## $ \text{Heston Model Simulation } \& \text{ Application} $

$\text{Importing Libraries}$

In [18]:
import numpy as np
from numpy import log, exp, pi, real
from scipy.integrate import quad
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm


$\text{In the Heston model the dynamics of the stock price and the variance of the stock returns are given by:}\\[1.5em]
\frac{dS}{S} = \mu dt + \sqrt{v}dz_1,\\[1.5em]
dv = \kappa(\theta - v)dt + \sigma\sqrt{v}dz_2\\[1.5em]
\text{where }  z_1 \text{ and } z_2 \text{ are two correlated Brownian motions such that  A stochastic discount factor such as:}\\[1.5em]
\frac{d\Lambda}{\Lambda} = -r dt - \frac{\lambda_1}{\sqrt{v}}dz_1 - \frac{\kappa \lambda_2}{\sigma \sqrt{v}}dz_2\\[1.5em]
\text{yields the following risk-neutral dynamics for the stock price and the stochastic variance of the returns}\\[1.5em]
\frac{dS}{S} = (r - q)dt + \sqrt{v}dz_1^*, \\
dv = \kappa(\theta^* - v)dt + \sigma\sqrt{v}dz_2^*\\[1.5em]
\text{The price of a European call with strike price } K \text{ and maturity } T \text{ is} \\[1.5em]
C = S e^{-qT} P_1 - K e^{-rT} P_2,\\[1.5em]
\text{where} \\[1.5em]
P_1 = \frac{1}{2} + \frac{1}{\pi} \frac{1}{F} \int_0^{\infty} \Re \left( \frac{e^{i \phi \ln(K)} f(\phi - i, 0)}{i \phi} \right) d\phi, \\
P_2 = \frac{1}{2} + \frac{1}{\pi} \int_0^{\infty} \Re \left( \frac{e^{i \phi \ln(K)} f(\phi, 0)}{i \phi} \right) d\phi.$

$\text{Compute the joint characteristic function using the following equation.}\\
\text{In the code, S0 and V0 denote the current stock price and variance. The parameter theta is the risk-adjusted parameter that I denote } \theta^* \text{ in t.}$

$f(\phi, \varphi) = \exp\left( i(r - q)\phi T + \left(\delta - \frac{2\gamma}{\sigma^2}\right)\kappa \theta^* T + i\phi x + \delta v \right) 
\times \left( e^{-\gamma T} - \frac{\sigma^2}{2\gamma}(1 - e^{-\gamma T})(i\phi - \delta) \right)^{-2\kappa \theta^*/\sigma^2}
\times \exp\left( \frac{v(i\phi - \delta)}{e^{-\gamma T} - \frac{\sigma^2}{2\gamma}(1 - e^{-\gamma T})(i\phi - \delta)} \right)$

In [19]:
def CF_Heston(phi1, phi2, S0, V0, T, r, q, kappa, theta, sigma, rho):
    gamma = (kappa**2 + (1 - rho**2) * sigma**2 * phi1**2 + 1j * (sigma - 2 * kappa * rho) * sigma * phi1)**0.5
    delta = (kappa + gamma - 1j * rho * sigma * phi1) / sigma**2
    
    y = exp(
        1j * (r - q) * phi1 * T 
        + (delta - 2 * gamma / sigma ** 2) * kappa * theta * T 
        + 1j * phi1 * log(S0) 
        + delta * V0
        + (- 2 * kappa * theta / sigma ** 2) 
        * np.log(exp(-gamma * T) - sigma**2 / (2 * gamma) * (1 - exp(- gamma * T)) * (1j * phi2 - delta))
        + V0 * (1j * phi2 - delta) / (exp(- gamma * T) - sigma**2 / (2 * gamma) * (1 - exp(- gamma * T)) * (1j * phi2 - delta)))
    return y

$\text{Next, compute } P_2 \text{ which denotes the risk-neutral probability that the call finishes in the money. Since there is no closed-form solution for this integral, we use the function quad from Scipy.}$

In [20]:
def P2(S0, V0, K, T, r, q, kappa, theta, sigma, rho):
    y = 0.5 + 1 / pi * quad(lambda phi1: real(exp(- 1j * phi1 * log(K)) * CF_Heston(phi1, 0, S0, V0, T, r, q, kappa, theta, sigma, rho) / (1j * phi1)), 0, np.inf, full_output=1)[0]
    return y

$\text{Similarly, compute } P_1, \text{ which allows us to compute the } \Delta \text{ of the call.}$

In [21]:
def P1(S0, V0, K, T, r, q, kappa, theta, sigma, rho):
    F = S0 * exp((r - q) * T)
    y = 0.5 + 1 / pi / F * quad(lambda phi1: real(exp(- 1j * phi1 * log(K)) * CF_Heston(phi1 - 1j, 0, S0, V0, T, r, q, kappa, theta, sigma, rho) / (1j * phi1)), 0, np.inf, full_output=1)[0]
    return y

$\text{ The value of a European call is then computed using: }$

In [22]:
def heston_call_price(S0, V0, K, T, r, q, kappa, theta, sigma, rho):
    call = S0 * exp(-q * T) * P1(S0, V0, K, T, r, q, kappa, theta, sigma, rho) - K * exp(-r * T) * P2(S0, V0, K, T, r, q, kappa, theta, sigma, rho)
    return call

$\text{Simulating for under the risk-neutral measure would allow us to compute:}\\[1.5em]
f(\phi, \varphi) = \mathbb{E}\left(e^{i\phi x(T) + \varphi v(T)}\right), \\
C(S, v) = \mathbb{E}\left(\max(S(T) - K, 0)e^{-rT}\right).$

$\text{First application is to find } \kappa, \theta, \sigma, \rho \text{, and }  v_0  \text{ for apple option prices that minimizes the root mean-squared error (RMSE):} \\[1.5em]
\text{RMSE} = \sqrt{\frac{1}{20} \sum_{i=1}^{5} \sum_{j=1}^{4} \left(C^{\text{obs}}(K_i, T_j) - C^{\text{model}}(K_i, T_j)\right)^2}$

### $\text{AAPL}$

In [23]:
market_strikes = np.array([200, 210, 220, 230, 240, 200, 210, 220, 230, 240, 
                           200, 210, 220, 230, 240, 200, 210, 220, 230, 240])

market_expirations = np.array([0.25, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 
                               0.75, 0.75, 0.75, 0.75, 0.75, 1, 1, 1, 1, 1])

market_prices = np.array([30.50, 26.40, 22.50, 9.55, 5.35, 35.10, 27.69, 20.96, 15.20, 10.48, 
                          39.01, 31.80, 25.30, 19.85, 15.15, 41.61, 35.57, 28.95, 23.50, 18.27])



def objective_function(params):
    kappa, theta, sigma, rho, V0 = params
    S0 = 226.21  
    r = 0.05  # 
    q = 0.005  

    model_prices = []
    for K, T in zip(market_strikes, market_expirations):
        model_price = heston_call_price(S0, V0, K, T, r, q, kappa, theta, sigma, rho)
        model_prices.append(model_price)
        
    model_prices = np.array(model_prices)
    rmse = np.sqrt(np.mean((market_prices - model_prices) ** 2))
    return rmse

initial_params = [1.0, 0.005, 0.005, 0, 0.01]

bounds = [(0.001, 5), (0.001, 1), (0.001, 2), (-1, 1), (0.001, 1)]

result = minimize(objective_function, initial_params, bounds=bounds, method='L-BFGS-B')

print("Optimized parameters:")
print(f"kappa: {result.x[0]:.4f}, theta: {result.x[1]:.4f}, sigma: {result.x[2]:.4f}, rho: {result.x[3]:.4f}, V0: {result.x[4]:.4f}")
print(f"Minimum RMSE: {result.fun:.4f}")


Optimized parameters:
kappa: 5.0000, theta: 0.0409, sigma: 0.7543, rho: -0.7857, V0: 0.1087
Minimum RMSE: 1.5063


In [24]:
S0 = 226.21  
r = 0.05  
q = 0.005
model_prices_opt = [heston_call_price(S0, result.x[4], K, T, r, q, result.x[0], result.x[1], result.x[2], result.x[3])
                    for K, T in zip(market_strikes, market_expirations)]

In [25]:
kappa = result.x[0]
theta = result.x[1]
sigma = result.x[2]
rho = result.x[3]
V0 = result.x[4]

# Market Data
S0 = 226.21  
r = 0.05  
market_strikes = np.array([200, 210, 220, 230, 240, 200, 210, 220, 230, 240, 
                           200, 210, 220, 230, 240, 200, 210, 220, 230, 240])

market_expirations = np.array([0.25, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 
                               0.75, 0.75, 0.75, 0.75, 0.75, 1, 1, 1, 1, 1])

market_prices = np.array([30.50, 26.40, 22.50, 9.55, 5.35, 35.10, 27.69, 20.96, 15.20, 10.48, 
                          39.01, 31.80, 25.30, 19.85, 15.15, 41.61, 35.57, 28.95, 23.50, 18.27])

def heston_prices(S0, K, T, r, q, kappa, theta, sigma, rho, V0):
    heston_price_list = []
    for strike, expiration in zip(K, T):
        price = heston_call_price(S0, V0, strike, expiration, r, q, kappa, theta, sigma, rho)
        heston_price_list.append(price)
    return np.array(heston_price_list)

heston_model_prices = heston_prices(S0, market_strikes, market_expirations, r, 0.005, kappa, theta, sigma, rho, V0)

def black_scholes_call(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def implied_volatility_objective(sigma, S0, K, T, r, model_price):
    bs_price = black_scholes_call(S0, K, T, r, sigma)
    return (bs_price - model_price) ** 2

def implied_volatility(S0, K, T, r, model_price):
    result = minimize(implied_volatility_objective, 0.2, args=(S0, K, T, r, model_price), bounds=[(0.001, 5)], method='L-BFGS-B')
    return result.x[0]

implied_vols = []
for K, T, model_price in zip(market_strikes, market_expirations, heston_model_prices):
    vol = implied_volatility(S0, K, T, r, model_price)
    implied_vols.append(vol)


$\text{AAPL Results Display}$

In [26]:
df = pd.DataFrame({
    'Market_Strikes': market_strikes,
    'Market_Expirations': market_expirations,
    'Market_Prices': market_prices,
    'Model_Prices_Opt': model_prices_opt,
    'Implied_Volatilities': implied_vols
})
df

,Market_Strikes,Market_Expirations,Market_Prices,Model_Prices_Opt,Implied_Volatilities
0,200,0.25,30.50,32.083147,0.308318
1,210,0.25,26.40,24.258231,0.293909
2,220,0.25,22.50,17.285968,0.278368
3,230,0.25,9.55,11.378711,0.262030
4,240,0.25,5.35,6.729932,0.245283
5,200,0.50,35.10,36.163002,0.269999
6,210,0.50,27.69,28.655798,0.258698
7,220,0.50,20.96,21.842445,0.246837
8,230,0.50,15.20,15.860246,0.234695
9,240,0.50,10.48,10.837619,0.222519


$\text{V0 Comparison}$

In [27]:
print(np.sqrt(result.x[4]) , 'compared to' , '0.308318')

0.32962286511485855 compared to 0.308318


$\text{Second attempt to find } \kappa, \theta, \sigma, \rho \text{, and }  v_0  \text{ for MSFT option prices by choosing different strike prices and time-to-maturity.}$

### $\text{MSFT}$

In [28]:
import yfinance as yf
from datetime import datetime
import pandas as pd
from scipy.optimize import minimize

msft = yf.Ticker("MSFT")

current_price = msft.history(period="1d")['Close'][-1]
print(f"msft price: {current_price}")

expiration_dates = msft.options
print("list exp:", expiration_dates)

selected_strikes = []
selected_prices = []
selected_expirations = []
selected_time_to_maturity = []

for expiration in expiration_dates[2:14]:  
    options = msft.option_chain(expiration)
    call_options = options.calls  

    call_options['price_diff'] = abs(call_options['strike'] - current_price)
    sorted_calls = call_options.sort_values('price_diff')

    selected_strikes.extend(sorted_calls['strike'].values[:2]) 
    selected_prices.extend(sorted_calls['lastPrice'].values[:2]) 
    selected_expirations.extend([expiration] * 2)  

today = datetime.today()
for exp in selected_expirations:
    expiration_date = datetime.strptime(exp, "%Y-%m-%d")
    time_to_expiration = (expiration_date - today).days / 365.0  
    selected_time_to_maturity.extend([time_to_expiration])

df = pd.DataFrame({
    'Strike (K)': selected_strikes,
    'Price': selected_prices,
    'Time to Maturity (Years)': selected_time_to_maturity,
    'Expiration Date': selected_expirations, 
    'Current Price (S)': [current_price] * len(selected_strikes)  
})

df


/var/folders/vr/pxvx2d3s3pd5lz4_4zp2hm280000gn/T/ipykernel_78302/102517310.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = msft.history(period="1d")['Close'][-1]


msft price: 418.1600036621094
list exp: ('2024-10-25', '2024-11-01', '2024-11-08', '2024-11-15', '2024-11-22', '2024-11-29', '2024-12-20', '2025-01-17', '2025-02-21', '2025-03-21', '2025-04-17', '2025-05-16', '2025-06-20', '2025-08-15', '2025-09-19', '2025-12-19', '2026-01-16', '2026-06-18', '2026-12-18', '2027-01-15')


,Strike (K),Price,Time to Maturity (Years),Expiration Date,Current Price (S)
0,420.0,11.30,0.052055,2024-11-08,418.160004
1,415.0,14.35,0.052055,2024-11-08,418.160004
2,420.0,12.80,0.071233,2024-11-15,418.160004
3,415.0,15.50,0.071233,2024-11-15,418.160004
4,420.0,13.80,0.090411,2024-11-22,418.160004
5,415.0,16.50,0.090411,2024-11-22,418.160004
6,420.0,14.33,0.109589,2024-11-29,418.160004
7,415.0,17.00,0.109589,2024-11-29,418.160004
8,420.0,17.49,0.167123,2024-12-20,418.160004
9,415.0,19.80,0.167123,2024-12-20,418.160004


In [29]:
from numpy import log, exp, pi, real
from scipy.optimize import minimize
from scipy.stats import norm

market_strikes = df['Strike (K)'].values  
market_expirations = df['Time to Maturity (Years)'].values  
market_prices = df['Price'].values  

def objective_function(params):
    kappa, theta, sigma, rho, V0 = params
    S0 = current_price  
    r = 0.05  
    q = 0.005  

    model_prices = []
    for K, T in zip(market_strikes, market_expirations):
        model_price = heston_call_price(S0, V0, K, T, r, q, kappa, theta, sigma, rho)
        model_prices.append(model_price)
    
    model_prices = np.array(model_prices)
    rmse = np.sqrt(np.mean((market_prices - model_prices) ** 2))
    return rmse

initial_params = [0.7, 0.05, 0.5, -0.5, 0.05]  
bounds = [(0.001, 5), (0.001, 1), (0.001, 2), (-1, 1), (0.001, 1)]

result = minimize(objective_function, initial_params, bounds=bounds, method='L-BFGS-B')

print("Optimized parameters:")
print(f"kappa: {result.x[0]:.4f}, theta: {result.x[1]:.4f}, sigma: {result.x[2]:.4f}, rho: {result.x[3]:.4f}, V0: {result.x[4]:.4f}")
print(f"Minimum RMSE: {result.fun:.4f}")


Optimized parameters:
kappa: 0.1285, theta: 1.0000, sigma: 2.0000, rho: 0.2833, V0: 0.0917
Minimum RMSE: 0.7218


In [30]:
kappa = result.x[0]
theta = result.x[1]
sigma = result.x[2]
rho = result.x[3]
V0 = result.x[4]

S0 = current_price 
r = 0.05  
market_strikes = df['Strike (K)'].values  
market_expirations = df['Time to Maturity (Years)'].values 
market_prices = df['Price'].values  

def heston_prices(S0, K, T, r, q, kappa, theta, sigma, rho, V0):
    heston_price_list = []
    for strike, expiration in zip(K, T):
        price = heston_call_price(S0, V0, strike, expiration, r, q, kappa, theta, sigma, rho)
        heston_price_list.append(price)
    return np.array(heston_price_list)

heston_model_prices = heston_prices(S0, market_strikes, market_expirations, r, 0.005, kappa, theta, sigma, rho, V0)

def black_scholes_call(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price

def implied_volatility_objective(sigma, S0, K, T, r, model_price):
    bs_price = black_scholes_call(S0, K, T, r, sigma)
    return (bs_price - model_price) ** 2

def implied_volatility(S0, K, T, r, model_price):
    result = minimize(implied_volatility_objective, 0.2, args=(S0, K, T, r, model_price), bounds=[(0.001, 5)], method='L-BFGS-B')
    return result.x[0]

implied_vols = []
for K, T, model_price in zip(market_strikes, market_expirations, heston_model_prices):
    vol = implied_volatility(S0, K, T, r, model_price)
    implied_vols.append(vol)


In [31]:
S0 = current_price 
r = 0.05   
q = 0.005
model_prices_opt = [heston_call_price(S0, result.x[4], K, T, r, q, result.x[0], result.x[1], result.x[2], result.x[3])
                    for K, T in zip(market_strikes, market_expirations)]


$\text{MSFT Results Display}$

In [32]:
df = pd.DataFrame({
    'Market_Strikes': market_strikes,
    'Market_Expirations': market_expirations,
    'Market_Prices': market_prices,
    'Model_Prices_Opt': model_prices_opt,
    'Implied_Volatilities': implied_vols
})
df

,Market_Strikes,Market_Expirations,Market_Prices,Model_Prices_Opt,Implied_Volatilities
0,420.0,0.052055,11.30,10.385143,0.282371
1,415.0,0.052055,14.35,12.735038,0.276784
2,420.0,0.071233,12.80,12.118991,0.276016
3,415.0,0.071233,15.50,14.446521,0.270776
4,420.0,0.090411,13.80,13.599950,0.270678
5,415.0,0.090411,16.50,15.916348,0.265797
6,420.0,0.109589,14.33,14.916306,0.266191
7,415.0,0.109589,17.00,17.228233,0.261653
8,420.0,0.167123,17.49,18.287270,0.256458
9,415.0,0.167123,19.80,20.605148,0.252787


$\text{V0 Comparison}$

In [33]:
print(np.sqrt(result.x[4]) , 'compared to' , '0.282371')

0.3028823704805597 compared to 0.282371


### $\text{Comments on resulted parameters on AAPL and MSFT options}$

<div style="font-family: 'Times New Roman', Times, serif; font-size: 16px; line-height: 1;">
    <p><strong>Kappa</strong>: The fact that MSFT options have a much lower kappa than AAPL implies a much slower mean-reversion process and much more unpredictable and volatile variance.</p>
    <p><strong>Theta</strong>: Based on the data collected in this time period, MSFT has a higher theta, suggesting that MSFT options perform more volatile than AAPL options.</p>
    <p><strong>Sigma</strong>: Same as theta, a higher sigma suggests that MSFT options’ variance has a higher volatility than AAPL options’.</p>
    <p><strong>Rho</strong>: There is a major difference. AAPL shows a strong negative correlation, while MSFT shows a positive correlation. This suggests that for MSFT, as the stock price increases, its variance tends to increase as well, which is contrary to AAPL.</p>
    <p><strong>V0</strong>: MSFT options have a lower initial underlying volatility based on the data collected.</p>
</div>